In [38]:
import os, re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import json

In [2]:
URL_LIST_BASE = "https://www.dogbreedslist.info/all-dog-breeds/list_1_{}.html"  # {} in [1, 19]

In [53]:
def get_dog_list_page(n):
    r = requests.get(URL_LIST_BASE.format(n))
    soup = BeautifulSoup(r.content, 'html.parser')
    divs = soup.find_all('div', {'class': 'left'})
    dog_urls = [div.find_all('a')[0]["href"] for div in divs]
    return [(re.findall("/all-dog-breeds/(.*)?\.html", url)[0], url) for url in dog_urls]

def load_dog_list_page():
    return json.load(open("./dog_url_list.json", "rb"))

In [58]:
# URL_LIST_ALL_DOGS = []
# for i in range(1, 20):
#     URL_LIST_ALL_DOGS.extend(get_dog_list_page(i))
#     time.sleep(2)

# json.dump(URL_LIST_ALL_DOGS, open("./dog_url_list.json", "w+"))

In [59]:
URL_LIST_ALL_DOGS_loaded = load_dog_list_page()

In [171]:
def get_dog_info(n):
    """Gets breed-specific info.  Unfortunately, numbered by index."""
    dog = URL_LIST_ALL_DOGS_loaded[n]
    r = requests.get(dog[1])
    soup = BeautifulSoup(r.content, 'html.parser')

    info_table = soup.find_all('table', {'class': 'table-01'})
    characteristics_table = soup.find_all('table', {'class': 'table-02'})
    
    dog_classes = [[i for i in tr.contents if i != '\n'] for tr in characteristics_table[0].find_all("tr")][1:]
    characteristics = {dog_class[0].string: dog_class[1].p.contents[0][0] for dog_class in dog_classes}
    info = dict([[i.string for i in info_table[0].find_all("tr")[2:][i].contents if i != '\n'] for i in [0, 4]])
    dog_info = [info["Name"], {**characteristics, **{'Size': info["Size"]}}]

    return dog_info

In [174]:
# ALL_DOG_INFO = {}
# for i in range(len(URL_LIST_ALL_DOGS_loaded)):
#     try:
#         time.sleep(1)
#         gdi = get_dog_info(i)
#         ALL_DOG_INFO[gdi[0]] = gdi[1]
#     except Exception as e:
#         print(i, e)

332 list index out of range
369 list index out of range


In [198]:
## SPECIAL CASES DUE TO BAD HTML.

# dog = URL_LIST_ALL_DOGS_loaded[332]
# r = requests.get(dog[1])
# soup = BeautifulSoup(r.content, 'html.parser')

# info_table = soup.find_all('table', {'class': 'table-01'})
# characteristics_table = soup.find_all('table', {'class': 'table-02'})

# dog_classes = [[i for i in tr.contents if i != '\n'] for tr in characteristics_table[0].find_all("tr")][1:]
# characteristics = {dog_class[0].string: dog_class[1].p.contents[0][0] for dog_class in dog_classes}
# # info = dict([[i.string for i in info_table[0].find_all("tr")[2:][i].contents if i != '\n'] for i in [0, 4]])
# dog_info = {**characteristics, **{"Size": "Medium"}}
# ALL_DOG_INFO["American Husky"] = dog_info

# dog = URL_LIST_ALL_DOGS_loaded[369]
# r = requests.get(dog[1])
# soup = BeautifulSoup(r.content, 'html.parser')

# info_table = soup.find_all('table', {'class': 'table-01'})
# characteristics_table = soup.find_all('table', {'class': 'table-02'})

# dog_classes = [[i for i in tr.contents if i != '\n'] for tr in characteristics_table[0].find_all("tr")][1:]
# characteristics = {dog_class[0].string: dog_class[1].p.contents[0][0] for dog_class in dog_classes}
# # info = dict([[i.string for i in info_table[0].find_all("tr")[2:][i].contents if i != '\n'] for i in [0, 4]])
# dog_info = {**characteristics, **{"Size": "Medium"}}
# ALL_DOG_INFO["Mountain Cur"] = dog_info

In [200]:
json.dump(ALL_DOG_INFO, open("./dog_metadata.json", "w+"))